In [1]:
import torch
import numpy as np

In [23]:
size=tuple([2,3,2])
X=torch.randint(3,size=size)
A=torch.randint(3,size=size)
print("X mat")
print(X)
print("\n\nA mat")
print(A)

X mat
tensor([[[0, 1],
         [0, 1],
         [0, 2]],

        [[0, 0],
         [2, 0],
         [2, 2]]])


A mat
tensor([[[0, 2],
         [2, 2],
         [2, 0]],

        [[2, 1],
         [0, 1],
         [2, 0]]])


In [24]:
# out=torch.einsum('bnd,df->bnf',(X, A))
At=torch.transpose(A,1,2)
print(At)
out = torch.matmul(X,At)
print(out)

tensor([[[0, 2, 2],
         [2, 2, 0]],

        [[2, 0, 2],
         [1, 1, 0]]])
tensor([[[2, 2, 0],
         [2, 2, 0],
         [4, 4, 0]],

        [[0, 0, 0],
         [4, 0, 4],
         [6, 2, 4]]])
